In [2]:
import os
import pandas as pd

In [21]:
# Files are in 'domains' folder in the project. 
folder_path = './domains'

# Listing  all Excel files in the folder (each of the 8 domains still split by properties) 
excel_files = [
    f for f in os.listdir(folder_path)
    if f.endswith('.xlsx') or f.endswith('.xls')
]
print("Found Excel files:", excel_files)

Found Excel files: ['Art & Literature.xlsx', 'Demographics.xlsx', 'Entertainment.xlsx', 'Geography.xlsx', 'History.xlsx', 'Personalities.xlsx', 'Science.xlsx', 'Sports.xlsx']


In [22]:
# Reading each Excel file’s sheets (each property) into a nested dictionary
all_files_data = {}

for excel_file in excel_files:
    full_path = os.path.join(folder_path, excel_file)
    xls = pd.ExcelFile(full_path)
    sheet_dict = {}
    
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(full_path, sheet_name=sheet_name)
        sheet_dict[sheet_name] = df
    
    all_files_data[excel_file] = sheet_dict

In [23]:
# Doing a weighted sampling of 500 rows per Excel file, to account for "not as populated" properties.
final_dataframes = []

for excel_file, sheets_dict in all_files_data.items():
    # Calculating the total rows across all sheets
    sheet_sizes = {s: len(df) for s, df in sheets_dict.items()}
    total_rows = sum(sheet_sizes.values())
    
    # Determining sampling sizes (weighted by row count)
    sample_sizes = {}
    for sheet_name, size in sheet_sizes.items():
        sample_sizes[sheet_name] = int(500 * size / total_rows)  # floor
    
    # Fix rounding to ensure total is exactly 500
    allocated = sum(sample_sizes.values())
    difference = 500 - allocated
    if difference != 0:
        # Identifying the largest sheet to fill up the remainder, if needed
        largest_sheet = max(sheet_sizes, key=sheet_sizes.get)
        sample_sizes[largest_sheet] += difference
    
    # Sampling from each sheet
    file_samples = []
    for sheet_name, df in sheets_dict.items():
        n_sample = sample_sizes[sheet_name]
        if n_sample > 0:
            # Ensuring it doesn’t sample more than the dataframe length
            n_sample = min(n_sample, len(df))
            
            # Adding a random_state for reproducibility. Use this if you want to achieve the exact same sample, otherwise remove the random_state.
            sampled_df = df.sample(n=n_sample, replace=False, random_state=17)
            
            # Adding columns to track origin (which category and which property)
            sampled_df['fileName'] = excel_file
            sampled_df['sheetName'] = sheet_name
            
            file_samples.append(sampled_df)
    
    # Combining all subsets for this file
    file_sampled_concatenated = pd.concat(file_samples, ignore_index=True)
    final_dataframes.append(file_sampled_concatenated)


In [24]:
# Stacking all 8 of the sampled datasets
final_df = pd.concat(final_dataframes, ignore_index=True)

In [25]:
# Adding a rowID column (ordered 1, 2, 3, ...)
final_df['rowID'] = range(1, len(final_df) + 1)

In [ ]:
# Checking the dataframe 
final_df.head()

In [19]:
# One more check for uniqueness of questions, in case duplicates fell through the cracks during the previous cleaning process
non_unique_questions = final_df['question'][final_df['question'].duplicated(keep=False)]
non_unique_questions

Series([], Name: question, dtype: object)

In [ ]:
# Remove ALL of these instances of non-unique questions
final_df = final_df[~final_df['question'].isin(non_unique_questions)]

# Display the updated dataset and verify
print("Remaining rows after removing non-unique questions:", final_df.shape[0])
final_df.head()

In [ ]:
# Adding a "days_since_last_modified" column, with reference date as the 1st of January, 2025

# Converting dateModified to do this calculation
df['dateModified'] = pd.to_datetime(df['dateModified'], dayfirst=True)

# Define the reference date (1st January 2025)
reference_date = pd.Timestamp('2025-01-01')

# Calculate the "days_since_last_modified" column
df['days_since_last_modified'] = (pd.Timestamp('2025-01-01') - df['dateModified']).dt.days

In [ ]:
# Keeping only the important columns for analysis.
final_df_clean = final_df[["rowID","entity","entityLabel","category","property", "propertyLabel", "value", "roundedValue", "unitLabel","sitelinks","dateModified","days_since_last_modified","entityType","question","fileName","sheetName"]]

In [ ]:
# Renaming 'category' as 'domain', for consistency
final_df_clean = final_df_clean.rename(columns={"category":"domain"})

In [26]:
# Confirming final shape 
# This should return about 4000 entries, as we have 8 Excel files (domains) with 500 random entries each. 
# If there are less, is because one domain didn't have enough.
print("Final DataFrame shape:", final_df_clean.shape)

Final DataFrame shape: (3932, 28)


In [27]:
# Saving to an Excel file
final_df_clean.to_csv('NumerFacts.csv', index=False)

In [28]:
# Getting the "questions" separately, to probe the models and avoid any sort of leak
if 'question' in final_df_clean.columns:
    final_df_clean[['question']].to_csv('NumerFacts_questions_only.csv', index=False)